# Deep COVID: An Exposition

Throughout our report, we present the contents of 'Deep-COVID: Predicting COVID-19 from chest X-ray images using deep transfer learning' (Deep-COVID, for short) by Minaee et al. We start with an introduction to the research question and overview of the data. Following this, we fit a logistic regression to the classification task. This will be used as our baseline model to compare the models used in the paper against. It also serves as an introduction to the basic concepts of Artificial Neural Networks (ANNs), as logistic regression can be viewed as a perceptron with sigmoid activation function. Using this as a segue, we, next, discuss the basic concepts involved in designing, and training ANN's. Once this is done, we introduce the concepts particular to Convolutional Neural Networks (CNNs). Then, we briefly discuss transfer learning and present the models used in the paper. Finally, we review their model evaluation metrics in comparison with eachother, our initial regression model, current biological-based tests, and discuss potential future work.

## The Goal of Deep-COVID and the Data

### The Goal

The COVID pandemic represents one of the biggest challenges currently facing the United States and the rest of the world. Before vaccination is sufficiently deployed and adopted, efforts towards treatment and prevention are of much importance. One of the most effective tools we have at our disposal is accurate and speedy detection. Diagnosing cases from radiography and radiology image is, perhaps, one of the fastest forms of detection (Minaee et al., 2020). In their paper, Minaee et al. apply transfer learning to fit famous ANNs, including ResNet18, ResNet50, SqueezeNet, and DenseNet-121, to identify COVID-19 disease in analyzed chest X-ray images. Results were promising and it is the hope that these methods can be used to more accurately and quickly identify cases in exposed individuals. We'll discuss, in more detail, the concepts of transfer learning, as well as the basic concepts necessary to understand how the ANNs used are structured. First, however, let's become more acquainted with the data used for training and validation.

### The Data

...TBD

## Logistic Regression: A Baseline Model and Segue into ANNs

## Introduction to ANNs and Deep Learning

## Introduction to CNNs

## Models Utilized in Minaee Et al.

## Evaluation Metrics and Future Work